# Project 001 UCSD Data Science Visualization Bootcamp
* Noreen Malik 😃
* Rene Venema("dutch") 🐱
* John Warlop 🦁
* Alex Wooten 🏄

😛😳😗😓🙉😸🙈🙊😽💀💢💥✨💏👫👄👃👀👛👛🗼🔮🔮🎄🎅👻

## Dependencies -- imports


In [12]:
import pandas as pd
from sodapy import Socrata #pip install socrata
import plotly.plotly as py #pip installl plotly
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import json


## Constants -- API Keys, etc

In [2]:
# Read in API(.gitignore file has entry for api_keys.json file)
API_endpoint = "https://data.cms.gov/resource/spmh-qjdv.json" #2015 data


IRS_DATAFRAME_PICKLE = 'df_irs.pkl'
with open('api_keys.json') as json_data:
    api_keys = json.load(json_data)

# Socrata Section
socrata_domain = "data.cms.gov"
socrata_dataset_identifier = "spmh-qjdv"
socrata_token = api_keys['socrata']['App_Token']
#print(api_keys['socrata']['App_Token'])
#print(api_keys['socrata']['Secret_Token'])
#print(api_keys['zillow']['ZWSID'])



## Functions 


In [14]:
def return_irs_df(file_name):
    #return the pickled dataframe for irs data
    df_irs = pd.read_pickle(file_name)
    return(df_irs)

# IRS Dataframe
[Link to Documentation](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2015-zip-code-data-soi) |
[SOI Tax Stats](https://www.irs.gov/statistics/soi-tax-stats-statistical-methodology) | [Data Cleaning of IRS Data](https://docs.google.com/document/d/1kNd1S_Ufgy0pIjJSjLDHba8y-pK8OYe47iAGeamH04E/edit?usp=sharing)

In [15]:
# Dataframe columns are: ST(State), Zip(Zip Code), AGI(Adjusted Gross Income),TtlR(Total Returns)
# TtlSR(Total Single Returns), TtlJR(Total Joint Returns), Eld(Number of Elderly Returns -- over 60 years old)
# AGI column is agreegated: 
# 1=> [1-25000), 2=> [25000-50000), 3=> [50000-75000), 4=> [75000-100000),
# 5=> [100000,200000), 6=> [200000-]
df_irs = return_irs_df(IRS_DATAFRAME_PICKLE)
df_irs.head()

,ST,Zip,AGI,TtlR,TtlSR,TtlJR,Eld
0,AL,0,1,836320.0,481570.0,109790.0,149910.0
1,AL,0,2,494830.0,206630.0,146250.0,107480.0
2,AL,0,3,261250.0,80720.0,139280.0,73400.0
3,AL,0,4,166690.0,28510.0,124650.0,49990.0
4,AL,0,5,212660.0,19520.0,184320.0,65870.0


# Dataframe - Medicare csv/api (dutch)
[Link to Documentation](https://dev.socrata.com/foundry/data.cms.gov/ehrv-m9r6)


In [16]:
#main_df = pd.read_csv("main_data_cleaned.csv")
#main_df.head()
client = Socrata(socrata_domain, socrata_token)
results = client.get(socrata_dataset_identifier, limit = 100000000) #default for limit (integer) = 1000
main_df = pd.DataFrame.from_dict(results)
main_df.head()
# Convert column datatypes
## Convert to float
col1 = ["average_covered_charges","average_medicare_payments","average_total_payments"]
main_df[col1] = main_df[col1].astype("float")
## Convert to integer
col2 = ["provider_id","provider_zip_code","total_discharges"]
main_df[col2] = main_df[col2].astype("int")
# Export data to csv file
main_df.to_csv("main data cleaned.csv", encoding = "utf-8-sig", index = False)

In [21]:
# Import data from csv
main_df = pd.read_csv("main data cleaned.csv")
main_df.head()

,average_covered_charges,average_medicare_payments,average_total_payments,drg_definition,hospital_referral_region_hrr_description,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,1014783.45,141193.91,171081.18,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,AL - Birmingham,BIRMINGHAM,10033,UNIVERSITY OF ALABAMA HOSPITAL,AL,619 SOUTH 19TH STREET,35233,11
1,382051.00,167511.96,194081.39,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,AZ - Phoenix,PHOENIX,30103,MAYO CLINIC HOSPITAL,AZ,5777 EAST MAYO BOULEVARD,85054,28
2,971881.48,294419.62,324050.71,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA - San Diego,SAN DIEGO,50025,UNIVERSITY OF CALIFORNIA SAN DIEGO MEDICAL CENTER,CA,200 WEST ARBOR DRIVE,92103,21
3,1517857.61,233532.83,257177.33,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA - San Diego,SAN DIEGO,50100,SHARP MEMORIAL HOSPITAL,CA,7901 FROST ST,92123,18
4,1470341.06,338472.00,346150.12,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA - Sacramento,SACRAMENTO,50108,"SUTTER MEDICAL CENTER, SACRAMENTO",CA,2825 CAPITOL AVENUE,95816,17
5,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10023,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,AL - Montgomery,67,16920.79,6653.80,5374.14
6,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10029,EAST ALABAMA MEDICAL CENTER AND SNF,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,AL - Birmingham,51,11977.13,5834.74,4761.41
7,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10033,UNIVERSITY OF ALABAMA HOSPITAL,619 SOUTH 19TH STREET,BIRMINGHAM,AL,35233,AL - Birmingham,32,35841.09,8031.12,5858.50
8,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10039,HUNTSVILLE HOSPITAL,101 SIVLEY RD,HUNTSVILLE,AL,35801,AL - Huntsville,135,28523.39,6113.38,5228.40
9,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10040,GADSDEN REGIONAL MEDICAL CENTER,1007 GOODYEAR AVENUE,GADSDEN,AL,35903,AL - Birmingham,34,75233.38,5541.05,4386.94


# Avg Cost Per Provider (Alex)

In [22]:
# ACC = Average Covered Charges
#     The provider's average charge for services covered by Medicare for all discharges in the DRG. These will vary from
#     hospital to hospital because of differences in hospital charge structures.
# ATP = Average Total Payments
#     The average of Medicare payments to the provider for the DRG including the DRG amount, teaching,  disproportionate 
#     share, capital, and outlier payments for all cases. Also included are co-payment and deductible amounts that the 
#     patient is responsible for (average_medicare_payments)
# AMP = Average Medicare Payments
#     average_medicare_payments_2?
#     Not sure what this column represents


# Create a Groupby based on Provider Id and Provider name
# Calculate mean values in ACC, ATP, AMP (or we may just be interested in what the patient pays)
# I could also sum up the average value of payments by patient and by insurance to see average total value provider charges

cost_df = main_df[["provider_id", "provider_name", "average_covered_charges",\
                   "average_total_payments", "average_medicare_payments"]]

provider_group = cost_df.groupby(["provider_id", "provider_name"])
avg_cost_group = round(provider_group.mean(),2)
avg_cost_group

,,average_covered_charges,average_total_payments,average_medicare_payments
provider_id,provider_name,,,
10001,SOUTHEAST ALABAMA MEDICAL CENTER,44393.74,9921.48,8708.05
10005,MARSHALL MEDICAL CENTER SOUTH,17973.07,7479.25,6222.37
10006,ELIZA COFFEE MEMORIAL HOSPITAL,49021.15,9735.52,8436.42
10007,MIZELL MEMORIAL HOSPITAL,12576.78,5426.64,4587.29
10008,CRENSHAW COMMUNITY HOSPITAL,15928.86,6157.16,5122.02
10011,ST VINCENT'S EAST,46784.18,10322.22,9006.06
10012,DEKALB REGIONAL MEDICAL CENTER,39501.27,5893.77,4751.40
10016,SHELBY BAPTIST MEDICAL CENTER,49006.74,9257.32,7674.07
10018,CALLAHAN EYE HOSPITAL,31023.69,10373.69,8139.31


# Avg Cost of Procedure in each state(John)

In [25]:
#main_df['ACC'].groupby(main_df['Provider State']).head()
#state_office = df.groupby(['state', 'office_id']).agg({'sales': 'sum'})
#main_df.rename(columns = {list(main_df)[8]: 'TtlDis'}, inplace = True)
df_costs = main_df.groupby(['provider_state']).\
            aggregate({'average_covered_charges':'sum',\
                       'average_total_payments':'sum','total_discharges':'sum'})
df_costs['Ttl_billed_per_p']  = (df_costs['average_covered_charges']+df_costs['average_total_payments'])\
                                /df_costs['total_discharges']
df_costs['Ttl_covered_per_p'] =  df_costs['average_covered_charges']/df_costs['total_discharges']
df_costs['Ttl_oop_per_p'] = df_costs['average_total_payments']/df_costs['total_discharges']
df_costs.head(12)
#df_acc = main_df.groupby(['Provider State']).aggregate({'ACC':'sum'})
#df_atp = main_df.groupby(['Provider State']).aggregate({'ATP':'sum'})
#df_ttlD = main_df.groupby(['Provider State']).aggregate({'TtlDis':'sum'})
#df_ttlD.iloc[:,0:]

,average_covered_charges,average_total_payments,total_discharges,Ttl_billed_per_p,Ttl_covered_per_p,Ttl_oop_per_p
provider_state,,,,,,
AK,1.967722e+07,5.030051e+06,7014,3522.565220,2805.420746,717.144474
AL,2.140469e+08,4.310629e+07,157744,1630.193161,1356.925749,273.267411
AR,9.778537e+07,2.625726e+07,91376,1357.496849,1070.142807,287.354043
AZ,2.142096e+08,4.721704e+07,111890,2336.461032,1914.465892,421.995140
CA,1.476417e+09,2.594942e+08,503269,3449.271242,2933.653951,515.617291
CO,1.529026e+08,3.024909e+07,70133,2611.490263,2180.179872,431.310392
CT,1.165064e+08,3.759063e+07,101868,1512.712789,1143.699625,369.013164
DC,5.110860e+07,1.416205e+07,26936,2423.174999,1897.408538,525.766461
DE,2.708536e+07,9.805910e+06,36396,1013.607781,744.185055,269.422726


# Comparing procedure costs vs income in zip code(John)

# Data Exploration

## Dutch

In [26]:
# Unique DRG definitions
main_df["drg_definition"].nunique()

563

In [27]:
# Unique HRR Descriptions
main_df["hospital_referral_region_hrr_description"].nunique()

306

In [28]:
# Unique providers
main_df["provider_id"].nunique()# Unique states (including the federal district D.C. (Washington D.C.))
main_df["provider_state"].nunique()

51

In [29]:
# Providers per state
main_df.groupby("provider_state")["provider_id"].nunique()

provider_state
AK      8
AL     83
AR     44
AZ     59
CA    294
CO     45
CT     29
DC      7
DE      6
FL    167
GA    100
HI     12
IA     34
ID     13
IL    126
IN     86
KS     53
KY     65
LA     86
MA     57
MD     47
ME     18
MI     94
MN     48
MO     74
MS     56
MT     13
NC     81
ND      7
NE     24
NH     13
NJ     64
NM     27
NV     21
NY    151
OH    128
OK     81
OR     34
PA    152
RI     11
SC     54
SD     19
TN     92
TX    285
UT     31
VA     74
VT      6
WA     48
WI     65
WV     29
WY     10
Name: provider_id, dtype: int64

In [30]:
# US state map
## Total ACC, ATP and AMP by provider state
state_totals_df = main_df.groupby("provider_state", as_index = False).agg({"provider_id":"nunique",
                                                                           "average_covered_charges":"sum",
                                                                           "average_medicare_payments":"sum",
                                                                           "average_total_payments":"sum"})

cols = ["average_covered_charges","average_medicare_payments","average_total_payments"]
state_totals_df[cols] = state_totals_df[cols]/1000000

state_totals_df = state_totals_df.rename(columns={"provider_state":"State",
                                                  "provider_id":"Total providers",
                                                  "average_covered_charges":"ACC Total",
                                                  "average_medicare_payments":"AMP Total",
                                                  "average_total_payments":"ATP Total"
                                                 })

state_totals_df.head()

for col in state_totals_df.columns:
    state_totals_df[col] = state_totals_df[col].astype(str)

In [31]:
scl = [[0.0, "rgb(242,240,247)"],[0.2, "rgb(218,218,235)"],[0.4, "rgb(188,189,220)"],
            [0.6, "rgb(158,154,200)"],[0.8, "rgb(117,107,177)"],[1.0, "rgb(84,39,143)"]]

In [32]:
state_totals_df['text'] = state_totals_df["State"] + '<br>' +\
    "ACC"+state_totals_df["ACC Total"]+' AMP '+state_totals_df["AMP Total"]+'<br>'+\
    'ATP '+state_totals_df["ATP Total"]

In [33]:
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = state_totals_df["State"],
        z = state_totals_df["Total providers"].astype(float),
        locationmode = 'USA-states',
        text = state_totals_df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = '2015 US Medicare inpatient charges and payments by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )